In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import * 

In [4]:
a = tf.constant(1.0)
a

<tf.Tensor 'Const:0' shape=() dtype=float32>

In [5]:
b = tf.constant(1.0)
b

<tf.Tensor 'Const_1:0' shape=() dtype=float32>

In [8]:
c = a + b
c

<tf.Tensor 'add_2:0' shape=() dtype=float32>

In [10]:
with tf.Session() as sess:
    c_ = sess.run(c)
    print(c_)

2.0


In [17]:
df = pd.read_csv("/data/bank-full.csv", sep=";")
y = np.where(df.y == "yes", 1, 0)
X = df.iloc[:, 0:-1]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 1)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [19]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.841435,-0.513906,1.347581,0.654165,-0.563213,-0.410057,-0.299354,1.900682,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,1.511034,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
1,-0.652995,-0.590185,1.347581,-0.572683,0.391184,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,2.741608,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
2,-0.841435,-0.353719,0.385408,-0.351385,-0.245081,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
3,-0.464554,-0.434130,-1.538940,-0.421268,0.391184,-0.410057,-0.299354,1.900682,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
4,-1.689418,-0.283797,-1.538940,-0.665862,-0.563213,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,-0.364749,9.817097,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004


In [48]:
n_x = X_train.shape[1]
n_y = 2
n_h = 100
learning_rate = 0.01
epochs = 100

X = tf.placeholder(dtype=tf.float32, shape=[None, n_x])
y = tf.placeholder(dtype=tf.int32, shape=[None])

print("X", X)
print("y", y)

y_oh = tf.one_hot(y, depth=n_y)
print("y_oh: ", y_oh)



W1 = tf.Variable(tf.truncated_normal(shape=[n_x, n_h], stddev=0.1))
print("W", W1)

b1 = tf.Variable(tf.zeros(shape=[n_h]))
print("b", b1)

Z1 = tf.matmul(X, W1) + b1
A1 = tf.nn.relu(Z1)


W2 = tf.Variable(tf.truncated_normal(shape=[n_h, n_y], stddev=0.1))
print("W2", W2)

b2 = tf.Variable(tf.zeros(shape=[n_y]))
print("b2", b2)

Z2 = tf.matmul(A1, W2) + b2

Z = Z2
print("Z", Z)

A = tf.nn.softmax(Z)
print("Activation:", A)

y_pred = tf.argmax(Z, axis = 1, output_type=tf.int32)
print("y_pred", y_pred)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y, y_pred), tf.float32))
print("Accuracy", accuracy)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_oh, logits=Z))

print("cost: ", cost)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        _, cost_, accuracy_ = sess.run([optimizer, cost, accuracy]
                                    , feed_dict={X: X_train, y: y_train})
        if i % (epochs // 20) == 0:
            print(accuracy_)
    test_accuracy_, y_pred_ = sess.run([accuracy, y_pred], feed_dict={X: X_test, y: y_test}) 
    print("Test accuracy: ", test_accuracy_)



X Tensor("Placeholder_52:0", shape=(?, 42), dtype=float32)
y Tensor("Placeholder_53:0", shape=(?,), dtype=int32)
y_oh:  Tensor("one_hot_23:0", shape=(?, 2), dtype=float32)
W <tf.Variable 'Variable_53:0' shape=(42, 100) dtype=float32_ref>
b <tf.Variable 'Variable_54:0' shape=(100,) dtype=float32_ref>
W <tf.Variable 'Variable_53:0' shape=(42, 100) dtype=float32_ref>
b <tf.Variable 'Variable_54:0' shape=(100,) dtype=float32_ref>
Z Tensor("add_26:0", shape=(?, 2), dtype=float32)
Activation: Tensor("Softmax_18:0", shape=(?, 2), dtype=float32)
y_pred Tensor("ArgMax_19:0", shape=(?,), dtype=int32)
Accuracy Tensor("Mean_25:0", shape=(), dtype=float32)
cost:  Tensor("Mean_26:0", shape=(), dtype=float32)
0.809429
0.88611877
0.8937024
0.9022656
0.90368754
0.90425634
0.9056151
0.9069738
0.90782696
0.90890133
0.91038644
0.91098684
0.91206115
0.91307235
0.91417825
0.91518945
0.9162954
0.91724336
0.9180333
0.9184441
Test accuracy:  0.90622234


In [49]:
metrics.accuracy_score(y_test, y_pred_)

0.9062223532881156

In [50]:
df_train = pd.read_csv("/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/data/MNIST/mnist_test.csv", header = None)
X_train = df_train.iloc[:, 1:].values
X_test = df_test.iloc[:, 1:].values
y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32))
X_test = scaler.transform(X_test.astype(np.float32))

In [51]:
n_x = X_train.shape[1]
n_y = 10
n_h = 100
learning_rate = 0.01
epochs = 100

X = tf.placeholder(dtype=tf.float32, shape=[None, n_x])
y = tf.placeholder(dtype=tf.int32, shape=[None])

print("X", X)
print("y", y)

y_oh = tf.one_hot(y, depth=n_y)
print("y_oh: ", y_oh)



W1 = tf.Variable(tf.truncated_normal(shape=[n_x, n_h], stddev=0.1))
print("W", W1)

b1 = tf.Variable(tf.zeros(shape=[n_h]))
print("b", b1)

Z1 = tf.matmul(X, W1) + b1
A1 = tf.nn.relu(Z1)


W2 = tf.Variable(tf.truncated_normal(shape=[n_h, n_y], stddev=0.1))
print("W2", W2)

b2 = tf.Variable(tf.zeros(shape=[n_y]))
print("b2", b2)

Z2 = tf.matmul(A1, W2) + b2

Z = Z2
print("Z", Z)

A = tf.nn.softmax(Z)
print("Activation:", A)

y_pred = tf.argmax(Z, axis = 1, output_type=tf.int32)
print("y_pred", y_pred)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y, y_pred), tf.float32))
print("Accuracy", accuracy)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_oh, logits=Z))

print("cost: ", cost)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        _, cost_, accuracy_ = sess.run([optimizer, cost, accuracy]
                                    , feed_dict={X: X_train, y: y_train})
        if i % (epochs // 20) == 0:
            print(accuracy_)
    test_accuracy_, y_pred_ = sess.run([accuracy, y_pred], feed_dict={X: X_test, y: y_test}) 
    print("Test accuracy: ", test_accuracy_)

X Tensor("Placeholder_54:0", shape=(?, 784), dtype=float32)
y Tensor("Placeholder_55:0", shape=(?,), dtype=int32)
y_oh:  Tensor("one_hot_24:0", shape=(?, 10), dtype=float32)
W <tf.Variable 'Variable_57:0' shape=(784, 100) dtype=float32_ref>
b <tf.Variable 'Variable_58:0' shape=(100,) dtype=float32_ref>
W2 <tf.Variable 'Variable_59:0' shape=(100, 10) dtype=float32_ref>
b2 <tf.Variable 'Variable_60:0' shape=(10,) dtype=float32_ref>
Z Tensor("add_28:0", shape=(?, 10), dtype=float32)
Activation: Tensor("Softmax_19:0", shape=(?, 10), dtype=float32)
y_pred Tensor("ArgMax_20:0", shape=(?,), dtype=int32)
Accuracy Tensor("Mean_27:0", shape=(), dtype=float32)
cost:  Tensor("Mean_28:0", shape=(), dtype=float32)
0.073533334
0.8743333
0.91185
0.92973334
0.9411167
0.9497333
0.9579167
0.96318334
0.9673333
0.9714
0.97461665
0.97688335
0.97925
0.98145
0.9834833
0.98518336
0.9865
0.98791665
0.98905
0.99023336
Test accuracy:  0.9695
